In [2]:
!pip install jinja2
!pip install pandas

## 0. 사용자별 전공/수강 과목 설정

자신의 전공 과목 코드를 적어주세요.
자신이 이전에 들었던 수강 과목 이름을 적어주세요. (이번 학기 시간표에서 제외하기 위함입니다. 재수강하고 싶으면 적지 마셔야 합니다.)

In [3]:
# =======================
# 사용자 선택 전공 (순서 중요)
# =======================
USER_MAJORS = [
    "KOR",  # 1전공 (국어국문학과 - 일반 전공)
    "BDS",  # 2전공 (빅데이터사이언스 - 연계 전공)
    "MGT"   # 3전공 (경영학과 - 일반 전공)
]

In [4]:
# ====================
# 이전 수강 과목 리스트
# ====================
TAKEN_COURSES = [
    "국문학개설", "문학이란 무엇인가", "국어학입문",
    "국어음운론", "국어형태론", "국어방언론", "국어의미론",
    "현대시텍스트읽기I", "영상문학론",
    "문학과 융합적 상상력",
    "고전소설텍스트읽기", "설화 문학의 이해"
    "선형대수학", 
    "경영통계학", "경제통계학", "경제통계학", "통계학입문",
    "기초빅데이터프로그래밍",
    "고급응용C프로그래밍", "기초C언어", "C언어기초", "C프로그래밍기초",
    "인공지능(딥러닝)개론", "웹데이터수집과 텍스트분석(캡스톤디자인)", "Data&AI",
    "회계학원론", "조직행동이론", "마케팅원론",
    "국제경영론", "경영전략",
    "의사결정론"
]

## 1. 환경 준비 & 이번 학기 설정

In [5]:
# ====================
# 환경 설정
# ====================
import re
from IPython.display import HTML
import pandas as pd

# 컬럼 안의 문자열 최대 표시 길이 설정 (None이면 제한 없음)
pd.set_option('display.max_colwidth', None)
# 출력할 컬럼 개수 제한 해제
pd.set_option('display.max_columns', None)
# 출력할 행 개수 제한 해제
pd.set_option('display.max_rows', None)


# ====================
# 이번 학기 설정
# ====================
SEMESTER = "2025-2"  # 바꾸면 자동 적용
DATA_PATH = f"data/{SEMESTER}-sis.xls"  # 학기별 시간표 HTML 파일 경로

# ====================
# 데이터 불러오기
# ====================
tables = pd.read_html(DATA_PATH, header=0)  # 첫 번째 행을 헤더로 읽기
df_all = tables[0]
df_all.head(3)


,학년도,학기,소속,학과,과목번호,분반,과목명,학점,수업시간/강의실,시간,교수진,수강생수,영어강의,중국어강의,승인과목,CU과목,홀짝구분,국제학생,Honors과목,공학인증,시험일자,수강대상,권장학년,수강신청 참조사항,과목 설명,비고,Unnamed: 26
0,2025 학년도,2학기,대학,게페르트국제학부,AAS1001,1,아시아학 개론,3.0,"화,목 12:00~13:15",3.0,김소연,0,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전학년,1-4학년,NaN,NaN,게페르트국제학부 필수 전공입문(Geppert School of International Studies Pre-Major)
1,2025 학년도,2학기,대학,게페르트국제학부,AAS2005,1,아시아역사의이해,3.0,"월,수 16:30~17:45",3.0,김종호,0,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전학년,1-4학년,NaN,NaN,아시아전공 전공필수선택(Asian Studies Program Major Core)
2,2025 학년도,2학기,대학,게페르트국제학부,AAS2025,1,현대 아시아의 중요 이슈들 Ⅱ,3.0,"월,수 10:30~11:45",3.0,양창원,0,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전학년,1-4학년,NaN,NaN,아시아전공 전공선택(Asian Studies Program Major Elective)


## 2. 과목코드 정의
필요하면 나중에 더 추가

In [6]:
# =======================
# 연계전공 코드 사전
# =======================
LINKED_MAJOR_CODES = {
    "BDS": {  # 빅데이터사이언스
        "codes": [
            "STS2011", "MAT2110", "MAT3020", "MGT2002", "ECO2004", 
            "BDS4010", "CSW4010", "CSE4187", "AIC4012", "BDS3010", "BDS3020",
            "CSE4130", "CSW2010", "CSW2020", "CSW2030", "CSE3080", "CSW2050",
            "CSW3010", "CSE3081", "CSW3030", "CSE4110", "CSW3060", "CSW3080",
            "CSW4020", "ECO2009", "ECO3022", "ECO3023", "ECO4003", "ECO4004",
            "ECO4032", "EEE4178", "JAS4014", "MAS1004", "MAS2009", "MAS2010",
            "MAT3110", "MAT4331", "MGT4202", "MGT4208", "MGT4226", "MGT4515",
            "MGT4517", "MGT6613", "MGTG613"
        ]
    },
    "PUB": {  # 공공인재
        "codes": [
            "PUB2005", "POL3130", "POL2002", "SOC2001", "SOC2003", "SOC3010",
            "ECO2001", "ECO2002", "ECO3009", "ECO3011", "ECO3017", "MGT2002",
            "MGT2003", "PHI2005", "PSY2001", "PSY3009", "PUB3030", "PUB3016",
            "PUB3029", "PUB3023", "PUB3024", "PUB3025", "PUB3031", "PUB3032",
            "PUB3026", "PUB3021", "PUB3020", "PUB3022", "PUB3028", "PUB3027",
            "PUB4009", "KOR4500", "EDU2001", "PHI4010", "ECO2007", "MGT3004",
            "MGT4301", "MGT3005", "MGT4404"
        ]
    },
    "EDU": {  # 교육학
        "codes": [
            "EDU2001", "EDU2002", "EDU2003", "EDU2004", "EDU3001", "EDU3047",
            "EDU3002", "EDU3033", "EDU3045", "EDU3046", "EDU3037", "EDU3004",
            "EDU3035", "EDU2005", "EDU2007", "EDU3038", "EDU3039", "EDU3048",
            "EDU3049", "SHU4019", "SHU4022", "SHU4031", "EDU3036"
        ]
    }
}


In [7]:
# =======================
# 전공별 과목 코드 set 생성 함수
# =======================
def get_major_codes(df_all, user_majors):
    major_code_sets = {}
    for major in user_majors:
        if major in LINKED_MAJOR_CODES:  # 연계전공
            # 연계전공: 지정된 과목 코드만 사용
            codes = set(df_all.loc[df_all['과목번호'].isin(LINKED_MAJOR_CODES[major]["codes"]), '과목번호'])
            major_code_sets[major] = codes
        else:  # 일반 전공
            codes = set(df_all.loc[df_all['과목번호'].str.startswith(major, na=False), '과목번호'])
            major_code_sets[major] = codes
    return major_code_sets

# 전공별 코드 set 생성
major_code_sets = get_major_codes(df_all, USER_MAJORS)

## 3. 시간표 생성

In [11]:
# =======================
# 과목명 전처리
# =======================
import re
import unicodedata
import pandas as pd

def clean_course_name(name):
    if pd.isna(name):
        return ""
    name = str(name)
    # 유니코드 정규화 (한글 자모/합성 문자 동일화)
    name = unicodedata.normalize("NFC", name)
    # 괄호 및 괄호 안 내용 제거
    name = re.sub(r"\(.*?\)", "", name)
    # 제로폭 문자 제거
    name = re.sub(r"[\u200B-\u200D\uFEFF]", "", name)
    # 비표준 공백(U+00A0 등) → 일반 공백
    name = re.sub(r"\s+", " ", name, flags=re.UNICODE)
    # 앞뒤 공백 제거
    return name.strip()

In [12]:
def make_timetable(df_all, semester, major_code_sets, taken_courses_names):
    # 0. 과목명 → 과목번호 매핑
    df_all['과목명_clean'] = df_all['과목명'].apply(clean_course_name)
    name_to_code = dict(zip(df_all['과목명_clean'], df_all['과목번호']))

    # 1. 이미 들은 과목(과목명) → 과목번호 변환
    taken_course_codes = {
        name_to_code.get(clean_course_name(name))
        for name in taken_courses_names
        if name_to_code.get(clean_course_name(name))
    }

    # 2. 학기 필터링
    year, term = semester.split("-")
    df_sem = df_all[
        df_all["학년도"].astype(str).str.contains(year) &
        df_all["학기"].astype(str).str.contains(term)
    ].copy()

    # 3. 전공 과목번호 필터링
    selected_codes = set().union(*(major_code_sets[m] for m in USER_MAJORS))
    df_sem = df_sem[df_sem["과목번호"].isin(selected_codes)]

    # 4. 이미 들은 과목 제외 (과목번호 기준)
    df_sem = df_sem[~df_sem["과목번호"].isin(taken_course_codes)]

    # 5. 긴 시간대 쪼개기
    split_times = {
        "09:00~11:45": ["09:00~10:15", "10:30~11:45"],
        "10:30~13:15": ["10:30~11:45", "12:00~13:15"],
        "13:30~16:15": ["13:30~14:45", "15:00~16:15"]
    }

    expanded_rows = []
    for _, row in df_sem.iterrows():
        time_room = str(row.get("수업시간/강의실", ""))
        days = re.findall(r"[월화수목금토일]", time_room)
        times = re.findall(r"\d{2}:\d{2}~\d{2}:\d{2}", time_room)

        if not days or not times:
            continue

        for day in days:
            for t in times:
                for time_item in split_times.get(t, [t]):
                    expanded_rows.append({
                        "시간": time_item,
                        "요일": day,
                        "과목번호": row["과목번호"],
                        "과목명": row["과목명"],
                        "교수진": row.get("교수진", "")
                    })

    expanded_df = pd.DataFrame(expanded_rows)
    if expanded_df.empty:
        return pd.DataFrame()

    timetable = (
        expanded_df.groupby(["시간", "요일"], as_index=False)
        .apply(lambda g: ", ".join(f"{r['과목명']}({r['교수진']})" for _, r in g.iterrows()))
        .reset_index(drop=True)
    )
    timetable.columns = ["시간", "요일", "과목(교수)"]

    요일순서 = ["월", "화", "수", "목", "금", "토", "일"]
    timetable_pivot = timetable.pivot(index="시간", columns="요일", values="과목(교수)").fillna("")
    timetable_pivot = timetable_pivot.reindex(columns=요일순서, fill_value="")

    return timetable_pivot


In [13]:
timetable = make_timetable(df_all, SEMESTER, major_code_sets, TAKEN_COURSES)
timetable

/var/folders/br/f3bzbxv50x36ysy4gwqml73h0000gn/T/ipykernel_63550/383811327.py:60: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: ", ".join(f"{r['과목명']}({r['교수진']})" for _, r in g.iterrows()))


요일,월,화,수,목,금,토,일
시간,,,,,,,
09:00~10:15,,"국어교과교재연구및지도법(강효경), 기술경영과기술사업화(백서현)","금융시장의빅데이터분석(캡스톤디자인)(정재식), 비즈니스프로세스관리(이준겸)","국어교과교재연구및지도법(강효경), 기술경영과기술사업화(백서현)","금융시장의빅데이터분석(캡스톤디자인)(정재식), 비즈니스프로세스관리(이준겸)",,
10:30~11:45,"빅데이터종합설계(캡스톤디자인)(하정욱), 자료구조(정성원), 자료구조입문(임종석), 문예창작론(장서란), 경영정보시스템(김용진), 재무관리(김도성), 기업윤리(장영균), 경영과학(민재형), 모바일통신경영(이상근), 인공지능과경영모델(김진화), 중급회계1(송민섭), 인공지능과마케팅(김주영)","알고리즘설계와분석(임인성), 데이터베이스입문(박석), 계량경제학I(남준우), 한문I(백진우), 리더쉽이론(이인석), 공급체인관리와e-business(김민균), 투자론(박영석), 원가회계(박예연), 기업과국제환경(김유진), 인공지능 자료분석(이윤동)","빅데이터종합설계(캡스톤디자인)(하정욱), 자료구조(정성원), 자료구조입문(임종석), 문예창작론(장서란), 경영정보시스템(김용진), 재무관리(김도성), 기업윤리(장영균), 경영과학(민재형), 모바일통신경영(이상근), 인공지능과경영모델(김진화), 중급회계1(송민섭), 인공지능과마케팅(김주영)","알고리즘설계와분석(임인성), 데이터베이스입문(박석), 계량경제학I(남준우), 한문I(백진우), 리더쉽이론(이인석), 공급체인관리와e-business(김민균), 투자론(박영석), 원가회계(박예연), 기업과국제환경(김유진), 인공지능 자료분석(이윤동)","블록체인및응용(문현아), 문학과문화콘텐츠(김예람), 기업과국제환경(김장순)",,
12:00~13:15,"알고리즘설계와분석(장형수), 기초Java언어(강호석), 계량경제학I(이성원), 국어의문장구조(이정훈), 고전문학자료연구(이정원), 선형대수학(최승일), 인적자원관리(조봉순), 노사관계론(양동훈), 디지털변혁과비즈니스모델혁신(김용진), 파생상품론(이정진), 중급회계2(송민섭)","현대소설텍스트읽기II(김경수), 국어교과논리및논술(강효경), 선형대수학(조장현), 산업수학(캡스톤디자인)(김종락), 경영정보시스템(오혜림), 운영관리(김길선), 관리회계(변상혁), 통계자료분석(이군희), 중급회계1(강평경), 소비자행동론(성연진)","알고리즘설계와분석(장형수), 기초Java언어(강호석), 계량경제학I(이성원), 국어의문장구조(이정훈), 고전문학자료연구(이정원), 선형대수학(최승일), 인적자원관리(조봉순), 노사관계론(양동훈), 디지털변혁과비즈니스모델혁신(김용진), 파생상품론(이정진), 중급회계2(송민섭)","현대소설텍스트읽기II(김경수), 국어교과논리및논술(강효경), 선형대수학(조장현), 산업수학(캡스톤디자인)(김종락), 경영정보시스템(오혜림), 운영관리(김길선), 관리회계(변상혁), 통계자료분석(이군희), 중급회계1(강평경), 소비자행동론(성연진)","블록체인및응용(문현아), 문학과문화콘텐츠(김예람), 기업과국제환경(김장순)",,
13:30~14:45,"산학프로젝트(캡스톤디자인)(장두성), 산학프로젝트(캡스톤디자인)(정영민), 세무회계(천준범), 국제마케팅론(김도의), 무역경영관리(손명옥)","알고리즘설계와분석(김세준), 데이터마이닝(임종섭), 한문과한문학(백진우), 운영관리(강세원), 금융기관론(이상호), 원가회계(박예연), 이해관계자자본주의와ESG(박영석), 로지스틱스와생산전략(하병천)","기초Java언어(강호석), 소설과영화(김예람), 선형대수학(김준태), 재무관리(홍광헌), 경영과학(김범수), 프로세스 수익경영(이준겸), 파생상품론(원재환), 서비스마케팅(이민경)","알고리즘설계와분석(김세준), 데이터마이닝(임종섭), 한문과한문학(백진우), 운영관리(강세원), 금융기관론(이상호), 원가회계(박예연), 이해관계자자본주의와ESG(박영석), 로지스틱스와생산전략(하병천)","기초Java언어(강호석), 국문학과전통문화(최지혜), 소설과영화(김예람), 선형대수학(김준태), 재무관리(홍광헌), 경영과학(김범수), 시뮬레이션(정철우), 프로세스 수익경영(이준겸), 파생상품론(원재환), 서비스마케팅(이민경)",,
14:00~17:00,,,,,회계/재무정보분석의이해와응용(이승영),,
15:00~16:15,"산학프로젝트(캡스톤디자인)(장두성), 산학프로젝트(캡스톤디자인)(정영민), 세무회계(천준범), 국제마케팅론(김도의), 무역경영관리(손명옥)","코퍼스언어학(홍정하), 계량경제학I(이영훈), 국어학연습(김한별), 이야기와이야기창작(김경수), 재무관리(이영주), 디지털 플랫폼 비즈니스(오혜림), 중급회계2(박두리), 유통전략론(임채운), 외환론(이강표), 창업의이해(캡스톤디자인)(김유진), 빅테크 가상 시장과 하이테크 마케팅(정재학), 경영학 연구에서의 경제적 모델링(김길선)","국어담화화용론(명정희), 운영관리(하병천), 관리회계(김민), 투자론(홍광헌), 금융리스크관리(원재환), 마케팅관리(nan)","코퍼스언어학(홍정하), 계량경제학I(이영훈), 국어학연습(김한별), 이야기와이야기창작(김경수), 재무관리(이영주), 디지털 플랫폼 비즈니스(오혜림), 중급회계2(박두리), 유통전략론(임채운), 외환론(이강표), 창업의이해(캡스톤디자인)(김유진), 빅테크 가상 시장과 하이테크 마케팅(정재학), 경영학 연구에서의 경제적 모델링(김길선)","국어담화화용론(명정희), 국문학과전통문화(최지혜), 운영관리(하병천), 관리회계(김민), 시뮬레이션(정철우), 투자론(홍광헌), 금융리스크관리(원재환), 마케팅관리(nan)",,
16:30~17:20,IT시대의 사회인준비특강(남궁훈),,,,,,
16:30~17:45,"문학이란무엇인가(박슬기), 선형대수학(박준용), 관리회계(양준선), 파생상품론(안성필), 외감법과 공인회계사법(nan)","고전시가론(최지혜), 경영과정보기술(김아현), 포트폴리오관리(이영주), 금융위기와 금융산업 혁신(이상호), 중국시장및경영환경의이해(이강표), 경영 데이터사이언스(조성빈)","문학이란무엇인가(박슬기), 선형대수학(박준용), 관리회계(양준선), 파생상품론(안성필), 외감법과 공인회계사법(nan)","고전시가론(최지혜), 경영과정보기술(김아현), 포트폴리오관리(이영주), 금융위기와 금융산업 혁신(이상호), 중국시장및경영환경의이해(이강표), 경영 데이터사이언스(조성빈)",,,


## 4. 시간표 좀 더 보기 좋게

In [14]:
import re
import pandas as pd

# =======================
# 0. 데이터 준비
# =======================
# df_all : 전체 강의 목록 DataFrame (컬럼: 과목명, 과목번호)
# timetable : 시간표 DataFrame (셀에 과목명 포함)
# USER_MAJORS : 사용자가 선택한 전공 리스트
# major_code_sets : 전공별 과목코드 set

# =======================
# 1. 전공별 색상 매핑 (HTML 색상명)
# =======================
COLOR_MAP = {
    "1전공": "purple",      # 보라
    "2전공": "blue",        # 파랑
    "3전공": "green",       # 초록
    "1&2전공": "red",       # 빨강
    "2&3전공": "orange",    # 주황
    "1&3전공": "deeppink"   # 진한 핑크
}


# =======================
# 2. name_to_code 매핑 생성
# =======================
df_all['과목명_clean'] = df_all['과목명'].apply(clean_course_name)
name_to_code = dict(zip(df_all['과목명_clean'], df_all['과목번호']))

# =======================
# 3. 전공 판별 함수 (수정 버전)
# =======================
def get_major_type(course_code):
    """
    과목번호를 기반으로 전공 구분을 반환.
    - 일반 전공: 과목코드 prefix로 판별
    - 연계전공: 미리 정의된 code 리스트로 판별
    반환 예: '1전공', '2전공', '1&2전공'
    """
    majors_found = set()  # 중복 방지를 위해 set 사용

    for idx, major in enumerate(USER_MAJORS, start=1):
        # 연계전공
        if major in LINKED_MAJOR_CODES:
            linked_codes = LINKED_MAJOR_CODES[major]["codes"]
            if course_code in linked_codes:
                majors_found.add(f"{idx}전공")
        # 일반 전공
        else:
            if course_code.startswith(major):
                majors_found.add(f"{idx}전공")

    if not majors_found:
        return None

    # 전공 번호 순서대로 정렬 후 문자열로 결합
    return "&".join(sorted(majors_found, key=lambda x: int(x[0])))


# =======================
# 4. HTML 컬러 적용 함수
# =======================
unmatched_courses = set()  # 매칭 안 된 과목 저장

def colorize_cell(cell):
    if pd.isna(cell) or cell.strip() == "":
        return cell

    courses = str(cell).split(', ')
    colored_courses = []

    for course_name in courses:
        course_name_clean = clean_course_name(course_name)

        # 과목코드 찾기
        course_code = name_to_code.get(course_name_clean, None)

        if not course_code:
            unmatched_courses.add(course_name_clean)
            colored_courses.append(course_name)
            continue

        # 전공 타입 찾기
        major_type = get_major_type(course_code)

        if not major_type or major_type not in COLOR_MAP:
            unmatched_courses.add(course_name_clean)
            colored_courses.append(course_name)
            continue

        # 색 적용
        color = COLOR_MAP[major_type]
        colored_courses.append(f'<span style="color:{color}">{course_name}</span>')

    return "<br>".join(colored_courses)


# =======================
# 5. 적용
# =======================
timetable_colored = timetable.map(colorize_cell)

from IPython.display import HTML
display(HTML(timetable_colored.to_html(escape=False)))

# =======================
# 6. 매칭 안 된 과목 확인
# =======================
if unmatched_courses:
    print("❗ 매칭 안 된 과목:")
    for c in sorted(unmatched_courses):
        print("-", c)


요일,월,화,수,목,금,토,일
시간,,,,,,,
09:00~10:15,,국어교과교재연구및지도법(강효경)기술경영과기술사업화(백서현),금융시장의빅데이터분석(캡스톤디자인)(정재식)비즈니스프로세스관리(이준겸),국어교과교재연구및지도법(강효경)기술경영과기술사업화(백서현),금융시장의빅데이터분석(캡스톤디자인)(정재식)비즈니스프로세스관리(이준겸),,
10:30~11:45,빅데이터종합설계(캡스톤디자인)(하정욱)자료구조(정성원)자료구조입문(임종석)문예창작론(장서란)경영정보시스템(김용진)재무관리(김도성)기업윤리(장영균)경영과학(민재형)모바일통신경영(이상근)인공지능과경영모델(김진화)중급회계1(송민섭)인공지능과마케팅(김주영),알고리즘설계와분석(임인성)데이터베이스입문(박석)계량경제학I(남준우)한문I(백진우)리더쉽이론(이인석)공급체인관리와e-business(김민균)투자론(박영석)원가회계(박예연)기업과국제환경(김유진)인공지능 자료분석(이윤동),빅데이터종합설계(캡스톤디자인)(하정욱)자료구조(정성원)자료구조입문(임종석)문예창작론(장서란)경영정보시스템(김용진)재무관리(김도성)기업윤리(장영균)경영과학(민재형)모바일통신경영(이상근)인공지능과경영모델(김진화)중급회계1(송민섭)인공지능과마케팅(김주영),알고리즘설계와분석(임인성)데이터베이스입문(박석)계량경제학I(남준우)한문I(백진우)리더쉽이론(이인석)공급체인관리와e-business(김민균)투자론(박영석)원가회계(박예연)기업과국제환경(김유진)인공지능 자료분석(이윤동),블록체인및응용(문현아)문학과문화콘텐츠(김예람)기업과국제환경(김장순),,
12:00~13:15,알고리즘설계와분석(장형수)기초Java언어(강호석)계량경제학I(이성원)국어의문장구조(이정훈)고전문학자료연구(이정원)선형대수학(최승일)인적자원관리(조봉순)노사관계론(양동훈)디지털변혁과비즈니스모델혁신(김용진)파생상품론(이정진)중급회계2(송민섭),현대소설텍스트읽기II(김경수)국어교과논리및논술(강효경)선형대수학(조장현)산업수학(캡스톤디자인)(김종락)경영정보시스템(오혜림)운영관리(김길선)관리회계(변상혁)통계자료분석(이군희)중급회계1(강평경)소비자행동론(성연진),알고리즘설계와분석(장형수)기초Java언어(강호석)계량경제학I(이성원)국어의문장구조(이정훈)고전문학자료연구(이정원)선형대수학(최승일)인적자원관리(조봉순)노사관계론(양동훈)디지털변혁과비즈니스모델혁신(김용진)파생상품론(이정진)중급회계2(송민섭),현대소설텍스트읽기II(김경수)국어교과논리및논술(강효경)선형대수학(조장현)산업수학(캡스톤디자인)(김종락)경영정보시스템(오혜림)운영관리(김길선)관리회계(변상혁)통계자료분석(이군희)중급회계1(강평경)소비자행동론(성연진),블록체인및응용(문현아)문학과문화콘텐츠(김예람)기업과국제환경(김장순),,
13:30~14:45,산학프로젝트(캡스톤디자인)(장두성)산학프로젝트(캡스톤디자인)(정영민)세무회계(천준범)국제마케팅론(김도의)무역경영관리(손명옥),알고리즘설계와분석(김세준)데이터마이닝(임종섭)한문과한문학(백진우)운영관리(강세원)금융기관론(이상호)원가회계(박예연)이해관계자자본주의와ESG(박영석)로지스틱스와생산전략(하병천),기초Java언어(강호석)소설과영화(김예람)선형대수학(김준태)재무관리(홍광헌)경영과학(김범수)프로세스 수익경영(이준겸)파생상품론(원재환)서비스마케팅(이민경),알고리즘설계와분석(김세준)데이터마이닝(임종섭)한문과한문학(백진우)운영관리(강세원)금융기관론(이상호)원가회계(박예연)이해관계자자본주의와ESG(박영석)로지스틱스와생산전략(하병천),기초Java언어(강호석)국문학과전통문화(최지혜)소설과영화(김예람)선형대수학(김준태)재무관리(홍광헌)경영과학(김범수)시뮬레이션(정철우)프로세스 수익경영(이준겸)파생상품론(원재환)서비스마케팅(이민경),,
14:00~17:00,,,,,회계/재무정보분석의이해와응용(이승영),,
15:00~16:15,산학프로젝트(캡스톤디자인)(장두성)산학프로젝트(캡스톤디자인)(정영민)세무회계(천준범)국제마케팅론(김도의)무역경영관리(손명옥),코퍼스언어학(홍정하)계량경제학I(이영훈)국어학연습(김한별)이야기와이야기창작(김경수)재무관리(이영주)디지털 플랫폼 비즈니스(오혜림)중급회계2(박두리)유통전략론(임채운)외환론(이강표)창업의이해(캡스톤디자인)(김유진)빅테크 가상 시장과 하이테크 마케팅(정재학)경영학 연구에서의 경제적 모델링(김길선),국어담화화용론(명정희)운영관리(하병천)관리회계(김민)투자론(홍광헌)금융리스크관리(원재환)마케팅관리(nan),코퍼스언어학(홍정하)계량경제학I(이영훈)국어학연습(김한별)이야기와이야기창작(김경수)재무관리(이영주)디지털 플랫폼 비즈니스(오혜림)중급회계2(박두리)유통전략론(임채운)외환론(이강표)창업의이해(캡스톤디자인)(김유진)빅테크 가상 시장과 하이테크 마케팅(정재학)경영학 연구에서의 경제적 모델링(김길선),국어담화화용론(명정희)국문학과전통문화(최지혜)운영관리(하병천)관리회계(김민)시뮬레이션(정철우)투자론(홍광헌)금융리스크관리(원재환)마케팅관리(nan),,
16:30~17:20,IT시대의 사회인준비특강(남궁훈),,,,,,
16:30~17:45,문학이란무엇인가(박슬기)선형대수학(박준용)관리회계(양준선)파생상품론(안성필)외감법과 공인회계사법(nan),고전시가론(최지혜)경영과정보기술(김아현)포트폴리오관리(이영주)금융위기와 금융산업 혁신(이상호)중국시장및경영환경의이해(이강표)경영 데이터사이언스(조성빈),문학이란무엇인가(박슬기)선형대수학(박준용)관리회계(양준선)파생상품론(안성필)외감법과 공인회계사법(nan),고전시가론(최지혜)경영과정보기술(김아현)포트폴리오관리(이영주)금융위기와 금융산업 혁신(이상호)중국시장및경영환경의이해(이강표)경영 데이터사이언스(조성빈),,,


❗ 매칭 안 된 과목:
- 인공지능 자료분석
- 코퍼스언어학
- 통계자료분석


- 아직 좀 부족함...